# 1. Problem Overview

#### Purpose

This analysis develops predictive models to estimate two company-level operational greenhouse gas emission targets consistent with the Greenhouse Gas Protocol. Accurate, auditable estimates enable downstream ESG reporting, portfolio-level exposure analysis, and risk assessment when disclosures are incomplete.

#### Definitions

- **Scope 1 — Direct emissions:** Emissions produced by sources owned or directly controlled by the reporting entity (e.g., combustion on site, company vehicle fleets, fugitive process emissions).
- **Scope 2 — Indirect emissions (purchased energy):** Emissions attributable to the generation of purchased electricity, heat or steam consumed by the company.

#### Why these targets are generally estimated

Many firms do not fully disclose emissions. Domain experts explain that financial institutions, corporates, and ESG analysts use modelled Scope 1 and Scope 2 values to fill these gaps for portfolio accounting, regulatory reporting, and basic risk assessment.

#### Why both targets are modelled separately

- Scope 1 reflects operational intensity and asset ownership; drivers include production scale, fuel use and sector technology.
- Scope 2 depends heavily on electricity consumption patterns and grid carbon intensity, which vary by geography and operations.

Practical implication: Models should be trained separately for each target while sharing thoughtfully engineered features where appropriate (e.g., revenue, sector exposure, adjusted ESG signals).


# 2. Target Distribution and ML Objectives

## 2.1 Observed distribution

- Both Scope 1 and Scope 2 show pronounced right skew and heavy tails. A handful of large industrial firms contribute a disproportionately large share of total emissions.
- Applying a log1p transform stabilizes variance and reduces skewness, producing distributions that are more amenable to standard supervised learners.

#### Implications for modelling:

- Consider log-based metrics for stability.
- Maintain separate models for Scope 1 and Scope 2 but use shared pre-processing conventions where appropriate (e.g., clipping and documented transforms).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load data
df = pd.read_csv('data/train.csv')

# Plot distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Scope 1
sns.histplot(df['target_scope_1'], kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Target Scope 1 Distribution')

# Scope 2
sns.histplot(df['target_scope_2'], kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Target Scope 2 Distribution')

# Log Scope 1
sns.histplot(np.log1p(df['target_scope_1']), kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Log(Target Scope 1) Distribution')

# Log Scope 2
sns.histplot(np.log1p(df['target_scope_2']), kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Log(Target Scope 2) Distribution')

plt.tight_layout()
plt.show()


## 2.2 Why Median-Focused Prediction

#### Rationale:

- Extreme emission values are frequently driven by asset-level characteristics (e.g., power plants, refineries) that are not observable in the provided dataset; attempting to fit those outliers closely will likely overfit.
- Stakeholders typically use aggregated or median estimates at portfolio or sector levels — therefore stability of central estimates is more valuable than perfect tail accuracy.
- RMSE is sensitive to a small number of extreme errors; MAE and log-MAE are more robust and align with multiplicative emission behavior.

## 2.3 Objective and Evaluation Metrics
Based on the distributional characteristics and practical use cases:
- **Primary objective:** Mean Absolute Error (MAE)
- **Secondary metrics:** Log-MAE, Log-RMSE

These design choices reduce sensitivity to heavy-tail noise, prioritize reliable central estimates, and align with the needs of typical users of emissions forecasts.


# 3. Feature Engineering — Overview

This section summarizes feature families, the rationale behind each group, and recommended transformations. The goal is to produce robust, interpretable predictors for Scope 1 and Scope 2 while avoiding implicit data leakage. A detailed feature evaluation is provided in the following section.

#### Feature families

1. Scale (Operational Size)
- Motivation: Revenue is a proxy for production scale and energy use.
- Candidate features: `revenue`, `log_revenue`, sector-partitioned absolute revenues, NACE revenue shares.
- Strategy: Provide both raw and log-transformed variants; derive scope-specific absolute revenues.

2. Geography
- Motivation: Under the Greenhouse Gas Protocol, both Scope 1 and Scope 2 vary significantly by country due to fuel mixes, industrial structure, and grid emission factors.
- Candidate features: `country_code` (categorical), region groups (collapse rare categories), country emission-factor lookups where available.
- Strategy: Use stable encodings (group rare categories, regularized target encoding for high-cardinality country codes).

3. Behavioural & Sustainability Signals
- Motivation: ESG scores, environmental activity adjustments and SDG commitments proxy for mitigation efforts and process maturity.
- Candidate features: adjusted `environmental_score`, `social_score`, `governance_score`, aggregated `env_score_adjustment`, SDG presence indicators.
- Strategy: Use adjusted environmental score rather than composite overall score to avoid multicollinearity; clip and standardize consistently across splits.



# 4. Scale of Business

## 4.1 Hypothesis 1

**Hypothesis:** Business scale, proxied by revenue, is one of the strongest predictors of operational emissions. Applying a log transformation to revenue significantly improves predictability.

### Evidence

- Both revenue and emissions are heavily right-skewed; applying log transformations produces more stable, bell-shaped distributions.
- Correlation analysis indicates a strong positive relationship between `log(revenue)` and `log(total emissions)` (correlation = 0.4291).

### Interpretation and Practical Takeaways

- Include `log_revenue` as a baseline feature; it stabilizes variance and improves performance for linear models.
- Scale is informative but not sufficient. Combining size features with sector exposure and geography provides better discrimination across firms.


In [ ]:
# Hypothesis 1: Revenue vs Emissions

# 1. Plot Revenue Distribution (Raw vs Log)
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.histplot(df['revenue'], kde=True, ax=axes[0])
axes[0].set_title('Revenue Distribution (Raw)')

sns.histplot(np.log1p(df['revenue']), kde=True, ax=axes[1])
axes[1].set_title('Revenue Distribution (Log)')

plt.tight_layout()
plt.show()

# 2. Calculate Log Correlations
df['log_revenue'] = np.log1p(df['revenue'])
df['log_total_emissions'] = np.log1p(df['target_scope_1'] + df['target_scope_2'])

correlation = df['log_revenue'].corr(df['log_total_emissions'])
print(f"Correlation between Log(Revenue) and Log(Total Emissions): {correlation:.4f}")


## 4.2 Hypothesis 2

**Hypothesis:** Sector-partitioned revenues provide additional, target-specific predictive power for Scope 1 and Scope 2.

### Procedure Summary

1. Map NACE sectors to scope-exposure categories (Scope 1, Scope 2, both, or neither) using conservative domain rules informed by expert guidance.
2. Compute absolute revenue per sector by multiplying total revenue by each sector’s revenue share.
3. Aggregate sector-level revenues into `scope_1_revenue` and `scope_2_revenue` buckets.
4. Generate log-transformed features and binary exposure flags:  
   - `log_scope_1_revenue`, `log_scope_2_revenue`  
   - `scope1_revenue_present`, `scope2_revenue_present`

### Evidence and Rationale

- Sectors with direct process emissions (manufacturing, mining, heavy industry) show stronger associations with Scope 1.
- Service, ICT, and office-based sectors correlate more with electricity use and thus Scope 2.
- Correlation analyses within non-zero subsets indicate that scope-partitioned revenues carry incremental predictive value beyond total revenue.
- Candle plots confirm that firms with no material sector revenue exhibit distinctively low emissions profiles.

### Interpretation and Practical Takeaways

- Include `log_scope_1_revenue` and `log_scope_2_revenue` as core features.
- Add binary materiality flags (`scope1_revenue_present`, `scope2_revenue_present`) to capture exposure even when revenue magnitudes are small.
- All subsequent analyses condition on company size by controlling for revenue before examining additional relationships.


In [ ]:
# Hypothesis 2: Sector Analysis

# Load sector data and classification
sector_df = pd.read_csv('data/revenue_distribution_by_sector.csv')
classification_df = pd.read_csv('data/sector_emission_scope_classification.csv')

# Merge classification
sector_df = pd.merge(sector_df, classification_df, on='nace_level_2_name', how='left')

# Fill missing classifications (if any) with False/True default (Scope 2 is universal)
sector_df['affects_scope_1'] = sector_df['affects_scope_1'].fillna(False)
sector_df['affects_scope_2'] = sector_df['affects_scope_2'].fillna(True)

# Create summary table
sector_summary = sector_df.groupby(['nace_level_1_code', 'nace_level_1_name', 'nace_level_2_name'])[['affects_scope_1', 'affects_scope_2']].first().reset_index()
sector_summary['company_count'] = sector_df.groupby(['nace_level_1_code', 'nace_level_1_name', 'nace_level_2_name']).size().values

# Display table
from IPython.display import display
display(sector_summary)

# Calculate Scope 1 / Scope 2 Revenue
merged_df = pd.merge(sector_df, df[['entity_id', 'revenue']], on='entity_id', how='inner')

# Calculate weighted revenue for each scope
# Note: A sector can contribute to BOTH Scope 1 and Scope 2
merged_df['scope_1_revenue_part'] = merged_df['revenue'] * merged_df['revenue_pct'] * merged_df['affects_scope_1'].astype(int)
merged_df['scope_2_revenue_part'] = merged_df['revenue'] * merged_df['revenue_pct'] * merged_df['affects_scope_2'].astype(int)

# Aggregate by entity
entity_revenue_split = merged_df.groupby('entity_id')[['scope_1_revenue_part', 'scope_2_revenue_part']].sum().reset_index()
entity_revenue_split.rename(columns={'scope_1_revenue_part': 'scope_1_revenue', 'scope_2_revenue_part': 'scope_2_revenue'}, inplace=True)

final_df = pd.merge(df, entity_revenue_split, on='entity_id', how='left').fillna(0)

# 1. Plot Scope 1 / Scope 2 Revenue Distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.histplot(final_df['scope_1_revenue'], kde=True, ax=axes[0, 0])
axes[0, 0].set_title('Scope 1 Revenue (Raw)')

sns.histplot(np.log1p(final_df['scope_1_revenue']), kde=True, ax=axes[0, 1])
axes[0, 1].set_title('Scope 1 Revenue (Log)')

sns.histplot(final_df['scope_2_revenue'], kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Scope 2 Revenue (Raw)')

sns.histplot(np.log1p(final_df['scope_2_revenue']), kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Scope 2 Revenue (Log)')

plt.tight_layout()
plt.show()

# 2. Calculate Log Correlations (Filtered for non-zero revenue)
final_df['log_scope_1_revenue'] = np.log1p(final_df['scope_1_revenue'])
final_df['log_scope_2_revenue'] = np.log1p(final_df['scope_2_revenue'])
final_df['log_target_scope_1'] = np.log1p(final_df['target_scope_1'])
final_df['log_target_scope_2'] = np.log1p(final_df['target_scope_2'])

# Filter for non-zero scope 1 revenue
df_s1 = final_df[final_df['scope_1_revenue'] > 0]
corr_s1 = df_s1['log_scope_1_revenue'].corr(df_s1['log_target_scope_1'])

# Filter for non-zero scope 2 revenue
df_s2 = final_df[final_df['scope_2_revenue'] > 0]
corr_s2 = df_s2['log_scope_2_revenue'].corr(df_s2['log_target_scope_2'])

print(f"Correlation (Filtered > 0): Log(Scope 1 Revenue) vs Log(Scope 1 Emissions): {corr_s1:.4f}")
print(f"Correlation (Filtered > 0): Log(Scope 2 Revenue) vs Log(Scope 2 Emissions): {corr_s2:.4f}")

# 3. Report Company Counts
has_s1 = (final_df['scope_1_revenue'] > 0).sum()
has_s2 = (final_df['scope_2_revenue'] > 0).sum()
has_both = ((final_df['scope_1_revenue'] > 0) & (final_df['scope_2_revenue'] > 0)).sum()
has_neither = ((final_df['scope_1_revenue'] == 0) & (final_df['scope_2_revenue'] == 0)).sum()

print(f"Companies with Scope 1 Revenue: {has_s1}")
print(f"Companies with Scope 2 Revenue: {has_s2}")
print(f"Companies with Both: {has_both}")
print(f"Companies with NO material sector revenue (Both False): {has_neither}")

# 4. Box Plots for Materiality Groups
final_df['materiality_group'] = 'Any Material'
final_df.loc[(final_df['scope_1_revenue'] == 0) & (final_df['scope_2_revenue'] == 0), 'materiality_group'] = 'None'

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(x='materiality_group', y='log_target_scope_1', data=final_df, ax=axes[0])
axes[0].set_title('Log(Scope 1 Emissions) by Materiality')

sns.boxplot(x='materiality_group', y='log_target_scope_2', data=final_df, ax=axes[1])
axes[1].set_title('Log(Scope 2 Emissions) by Materiality')

plt.tight_layout()
plt.show()


## 5. Geography

### 5.1 Hypothesis 3

**Hypothesis:** For companies with comparable scale and sector exposure, country-level factors explain residual differences in emissions. This effect is expected to be stronger for Scope 2 than for Scope 1.

### Motivation

Domain knowledge suggests that, under the Greenhouse Gas Protocol, both Scope 1 and Scope 2 vary meaningfully by country due to differences in fuel mixes, industrial structure, and grid emission factors. Geography therefore serves as a reasonable proxy for country-level determinants. We sought to validate this using the available data, even though high-quality country emission-factor tables were not included in the provided dataset.

### Analytical Approach

- Condition on company size by stratifying firms into revenue quantiles and comparing country distributions within each bin.  
- Apply Welch’s t-tests to evaluate mean differences and report multiplicative ratios in log space when informative.

### Expected Outcomes

- **Scope 1:** Only modest country-level differences, likely on the order of single-digit percent effects.  
- **Scope 2:** Larger variations driven by grid mix, potentially resulting in substantial multiplicative differences.

### Findings and Conclusions

**Country features**
- As a standalone feature, country shows limited signal due to high variance and small sample sizes in many categories.
- However, based on the definitions of Scope 1 and Scope 2, country-level factors should logically matter.
- The lack of strong detection is likely due to insufficient data rather than absence of effect; some countries with larger sample sizes do show statistically significant differences.
- Given the domain rationale and partial empirical support, country information is likely to improve performance in tree-based models (e.g., CatBoost).

**Region features**
- Aside from WEU and NAM, all other regions contain fewer than ten companies.
- Regions provide much coarser information than country codes and show weaker differentiation.

**Final Decision**
- **Use country as a categorical feature.**  
- **Exclude region features** due to sparsity and lower informational value.


In [ ]:
# Hypothesis 3: Country Analysis

import numpy as np
import pandas as pd
from scipy import stats

def analyze_country_factors(df, scope_col, revenue_col, country_col='country_code', 
                            n_quantiles=10, use_log_target=False):
    """
    use_log_target=False → 기존 raw target 분석
    use_log_target=True  → log1p(target) 기반 안정화된 factor 분석
    """
    
    df = df.copy()
    
    # Create log target column if needed
    if use_log_target:
        scope_col_used = f"log_{scope_col}"
        df[scope_col_used] = np.log1p(df[scope_col])
        print(f"\n=== Log-Transformed Analysis for {scope_col} vs {revenue_col} ===")
    else:
        scope_col_used = scope_col
        print(f"\n=== Analysis for {scope_col} vs {revenue_col} ===")
    
    # Step A: Filter and Quantiles
    df_valid = df[df[revenue_col] > 0].copy()
    df_valid['log_revenue'] = np.log1p(df_valid[revenue_col])
    
    # Quantile binning
    try:
        df_valid['quantile'] = pd.qcut(df_valid['log_revenue'], n_quantiles, labels=False)
    except ValueError:
        print("Warning: Not enough unique revenue values for quantiles. Using rank-based binning.")
        df_valid['quantile'] = pd.qcut(df_valid['log_revenue'].rank(method='first'), n_quantiles, labels=False)
    
    # Step B-D: Per group analysis
    for q in range(n_quantiles):
        group_df = df_valid[df_valid['quantile'] == q]
        if group_df.empty:
            continue
        
        country_counts = group_df[country_col].value_counts()
        if country_counts.empty:
            continue
        
        baseline_country = country_counts.idxmax()
        baseline_data = group_df[group_df[country_col] == baseline_country][scope_col_used]
        
        # Stats
        b_min = baseline_data.min()
        b_max = baseline_data.max()
        b_mean = baseline_data.mean()
        b_std = baseline_data.std()
        b_median = baseline_data.median()
        
        print(f"\n[Group {q} (Quantile {q+1}/{n_quantiles})]")
        print(f"- Baseline country: {baseline_country} (n={len(baseline_data)})")
        print(f"- Baseline stats: min={b_min:.4f}, max={b_max:.4f}, mean={b_mean:.4f}, std={b_std:.4f}, median={b_median:.4f}")
        
        results = []
        
        for country in country_counts.index:
            if country == baseline_country:
                continue
            
            c_data = group_df[group_df[country_col] == country][scope_col_used]
            if len(c_data) < 2:
                continue
            
            c_mean = c_data.mean()
            c_std = c_data.std()
            
            # Ratio changes if using log or raw
            if use_log_target:
                # In log space: exp(mean difference) gives multiplicative factor
                mean_ratio = np.exp(c_mean - b_mean)
            else:
                mean_ratio = c_mean / b_mean if b_mean != 0 else np.nan
            
            # Welch test
            t_stat, p_val = stats.ttest_ind(c_data, baseline_data, equal_var=False)
            is_sig = "Yes" if p_val < 0.05 else "No"
            
            results.append({
                'Country': country,
                'n': len(c_data),
                'Mean': c_mean,
                'Std': c_std,
                'Ratio': mean_ratio,
                'p-value': p_val,
                'Sig?': is_sig
            })
        
        if results:
            print(pd.DataFrame(results).to_string(index=False, float_format=lambda x: "{:.4f}".format(x)))
        else:
            print("No other countries with sufficient data for comparison.")


# Run Analysis for Scope 1
# Note: Using 'scope_1_revenue' calculated in previous step
analyze_country_factors(final_df, 'target_scope_1', 'scope_1_revenue')

# Run Analysis for Scope 2
# Note: Using 'scope_2_revenue' calculated in previous step
analyze_country_factors(final_df, 'target_scope_2', 'scope_2_revenue')

# Run Analysis for Log Scope 1
analyze_country_factors(final_df, 'target_scope_1', 'scope_1_revenue', use_log_target=True)

# Run Analysis for Log Scope 2
analyze_country_factors(final_df, 'target_scope_2', 'scope_2_revenue', use_log_target=True)



# 6. Behavioral Features

### Overall Score and E/S/G Scores

- `overall_score` is a weighted sum of the component scores (`0.45 * E + 0.30 * S + 0.25 * G`).  
  → Exclude due to multicollinearity with the underlying E/S/G features.  
- Scores are reported on a 1–5 scale, where lower values indicate better performance.  
- `data/environmental_activities.csv` provides additive adjustments to the environmental score (`env_score_adjustment`).

### Analytical Approach

- Compute correlations between emissions and the following features:  
  `overall_score`, `environmental_score`, `social_score`, `governance_score`, `env_adjusted_score`, `overall_adjusted_score`.
- Condition on company size by stratifying firms into revenue quantiles and re-computing correlations within each bin.

### Technical Conclusions — ESG and Adjusted Scores

- When ignoring scale, correlations between emissions and ESG scores appear small.
- After conditioning on revenue, several quantile groups show meaningful positive or negative correlations, indicating useful signal once scale effects are removed.
- The mixed positive/negative patterns across bins suggest a nonlinear relationship, which is well-suited for nonlinear models such as XGBoost.
- Adjusted scores (`env_adjusted_score`, `overall_adjusted_score`) consistently exhibit stronger correlations than their unadjusted counterparts.
- `overall_score` should not be used because it is a deterministic weighted sum of E/S/G components and introduces multicollinearity.

### Final Decision

- Use E/S/G component scores as features.  
- Prefer `env_adjusted_score` over the raw environmental score.  
- Do **not** use `overall_score` (or its adjusted version) due to multicollinearity.


In [ ]:
import numpy as np
import pandas as pd

# Load environmental activities
env_activities = pd.read_csv('data/environmental_activities.csv')

# Aggregate adjustments by entity_id
env_adj_agg = env_activities.groupby('entity_id')['env_score_adjustment'].sum().reset_index()

# Merge with final_df
# We use left merge to keep all companies in final_df, filling missing adjustments with 0
analysis_df = final_df.merge(env_adj_agg, on='entity_id', how='left')
analysis_df['env_score_adjustment'] = analysis_df['env_score_adjustment'].fillna(0)

# Calculate adjusted scores
analysis_df['env_adjusted'] = analysis_df['environmental_score'] + analysis_df['env_score_adjustment']
analysis_df['overall_adjusted'] = (0.45 * analysis_df['env_adjusted'] + 
                                   0.3 * analysis_df['social_score'] + 
                                   0.25 * analysis_df['governance_score'])

# Create log targets
analysis_df['log_target_scope_1'] = np.log1p(analysis_df['target_scope_1'])
analysis_df['log_target_scope_2'] = np.log1p(analysis_df['target_scope_2'])

# === Global Correlations (No Quantiles) ===
global_targets = [
    ('Scope 1 Target', 'target_scope_1'),
    ('Log Scope 1 Target', 'log_target_scope_1'),
    ('Scope 2 Target', 'target_scope_2'),
    ('Log Scope 2 Target', 'log_target_scope_2')
]

score_cols_map = {
    'overall_score': 'overall',
    'overall_adjusted': 'overall_adjusted',
    'environmental_score': 'env',
    'env_adjusted': 'env_adjusted',
    'social_score': 'social',
    'governance_score': 'governance'
}

global_results = []
for label, t_col in global_targets:
    row = {'target': label}
    for s_col, s_label in score_cols_map.items():
        # Calculate correlation on the full dataset (ignoring NaNs)
        row[s_label] = analysis_df[t_col].corr(analysis_df[s_col])
    global_results.append(row)

global_corr_df = pd.DataFrame(global_results)
# Ensure column order
global_cols = ['target', 'overall', 'overall_adjusted', 'env', 'env_adjusted', 'social', 'governance']
global_corr_df = global_corr_df[global_cols]

print("=== Global Correlations (All Data) ===")
print(global_corr_df.to_string(index=False, float_format=lambda x: "{:.4f}".format(x)))
print("\n")

# === Quantile Analysis ===
def calculate_quantile_correlations(df, revenue_col, target_col, title):
    score_cols = ['overall_score', 'overall_adjusted', 'environmental_score', 'env_adjusted', 'social_score', 'governance_score']
    
    # Filter for valid revenue
    valid_df = df[df[revenue_col] > 0].copy()
    
    # Create quantiles
    n_quantiles = 10
    try:
        valid_df['quantile'] = pd.qcut(valid_df[revenue_col], n_quantiles, labels=False)
    except ValueError:
        # Fallback if not enough unique values
        valid_df['quantile'] = pd.qcut(valid_df[revenue_col].rank(method='first'), n_quantiles, labels=False)
    
    # Calculate correlations per group
    results = []
    for q in range(n_quantiles):
        group_data = valid_df[valid_df['quantile'] == q]
        if len(group_data) < 2:
            continue
        
        row = {'quantile group': q}
        for col in score_cols:
            corr = group_data[col].corr(group_data[target_col])
            row[col] = corr
        results.append(row)
    
    # Create result dataframe
    corr_table = pd.DataFrame(results)
    
    # Rename columns for display
    corr_table = corr_table.rename(columns={
        'overall_score': 'overall',
        'environmental_score': 'env',
        'social_score': 'social',
        'governance_score': 'governance'
    })
    
    # Reorder columns
    display_cols = ['quantile group', 'overall', 'overall_adjusted', 'env', 'env_adjusted', 'social', 'governance']
    corr_table = corr_table[display_cols]
    
    print(f"=== {title} ===")
    print(corr_table.to_string(index=False, float_format=lambda x: "{:.4f}".format(x)))
    print("\n")

# 1. Scope 1 Revenue vs Target Scope 1
calculate_quantile_correlations(analysis_df, 'scope_1_revenue', 'target_scope_1', 
                              'Correlation: Scope 1 Revenue Quantile vs Target Scope 1')

# 2. Scope 1 Revenue vs Log Target Scope 1
calculate_quantile_correlations(analysis_df, 'scope_1_revenue', 'log_target_scope_1', 
                              'Correlation: Scope 1 Revenue Quantile vs Log Target Scope 1')

# 3. Scope 2 Revenue vs Target Scope 2
calculate_quantile_correlations(analysis_df, 'scope_2_revenue', 'target_scope_2', 
                              'Correlation: Scope 2 Revenue Quantile vs Target Scope 2')

# 4. Scope 2 Revenue vs Log Target Scope 2
calculate_quantile_correlations(analysis_df, 'scope_2_revenue', 'log_target_scope_2', 
                              'Correlation: Scope 2 Revenue Quantile vs Log Target Scope 2')


## 6.2 Sustainable Development Goals (SDGs)

### What are SDGs?
- SDGs reflect high-level sustainability commitments, such as:
  - SDG 7: Affordable and Clean Energy  
  - SDG 9: Industry, Innovation, and Infrastructure  
  - SDG 12: Responsible Consumption and Production  
  - SDG 13: Climate Action  

### Analytical Approach

- For each SDG, compare emissions between companies with and without the SDG commitment using Welch’s t-tests.
- Condition on company size by stratifying firms into revenue quantiles and repeat the tests within each bin.

### Technical Conclusions: SDGs do not provide predictive signal for emissions.
- Sample sizes are very small; no SDG pair shows consistent statistical significance.
- Effects visible at the global level disappear once controlling for revenue, indicating confounding (e.g., only small firms disproportionately reporting SDG commitments).
- SDGs are self-reported and often loosely connected to operational practices.
- The dataset is too sparse to avoid overfitting if encoded as features.

### Final Decision
- Do **not** include SDG-related features in the predictive model.


In [ ]:
import scipy.stats as stats

# Load SDG data
sdg_df = pd.read_csv('data/sustainable_development_goals.csv')
unique_sdgs = sdg_df[['sdg_id', 'sdg_name']].drop_duplicates().sort_values('sdg_id')

# Targets to analyze
targets = [
    ('target_scope_1', 'target scope 1'),
    ('log_target_scope_1', 'target scope 1 log'),
    ('target_scope_2', 'target scope 2'),
    ('log_target_scope_2', 'target scope 2 log')
]

significant_cases = []

for _, row in unique_sdgs.iterrows():
    sdg_id = row['sdg_id']
    sdg_name = row['sdg_name']
    
    print(f"[======={sdg_name} ({sdg_id})=======]")
    
    # Identify entities with this SDG
    entities_with_sdg = sdg_df[sdg_df['sdg_id'] == sdg_id]['entity_id'].unique()
    
    has_sdg = analysis_df['entity_id'].isin(entities_with_sdg)
    group_exist = analysis_df[has_sdg]
    group_non_exist = analysis_df[~has_sdg]
    
    # Global comparison table
    print("|target | N(exist) | N(non-exist) | existence mean | non existence mean | p-value | is significant|")
    for col, label in targets:
        a = group_exist[col].dropna()
        b = group_non_exist[col].dropna()
        
        n_exist = len(a)
        n_non_exist = len(b)
        
        if n_exist > 1 and n_non_exist > 1:
            stat, pval = stats.ttest_ind(a, b, equal_var=False)
            mean_exist = a.mean()
            mean_non_exist = b.mean()
            is_sig = pval < 0.05
        else:
            mean_exist = np.nan
            mean_non_exist = np.nan
            pval = np.nan
            is_sig = False
            
        print(f"|{label}| {n_exist} | {n_non_exist} | {mean_exist:.4f} | {mean_non_exist:.4f} | {pval:.4f} | {is_sig}|")
        
        if is_sig:
            significant_cases.append({
                'type': 'Global',
                'sdg': f"{sdg_name} ({sdg_id})",
                'target': label,
                'quantile': 'All',
                'p_value': pval,
                'mean_diff': mean_exist - mean_non_exist
            })
            
    print("\n")

    # Quantile Analysis Helper
    def run_quantile_analysis(revenue_col, target_col, target_label, quantile_label):
        print(f"[{quantile_label}, {target_label} mean comparison]")
        print("|quantile group| N(exist) | N(non-exist) | existence mean | non existence mean | p-value | is significant|")
        
        temp_df = analysis_df[analysis_df[revenue_col] > 0].copy()
        
        try:
            temp_df['quantile'] = pd.qcut(temp_df[revenue_col], 10, labels=False)
        except ValueError:
             temp_df['quantile'] = pd.qcut(temp_df[revenue_col].rank(method='first'), 10, labels=False)
             
        for q in range(10):
            q_data = temp_df[temp_df['quantile'] == q]
            
            has_sdg_q = q_data['entity_id'].isin(entities_with_sdg)
            g_exist = q_data[has_sdg_q][target_col].dropna()
            g_non_exist = q_data[~has_sdg_q][target_col].dropna()
            
            n_exist = len(g_exist)
            n_non_exist = len(g_non_exist)
            
            if n_exist > 1 and n_non_exist > 1:
                stat, pval = stats.ttest_ind(g_exist, g_non_exist, equal_var=False)
                m_exist = g_exist.mean()
                m_non_exist = g_non_exist.mean()
                is_sig = pval < 0.05
            else:
                m_exist = np.nan
                m_non_exist = np.nan
                pval = np.nan
                is_sig = False
            
            print(f"|{q}| {n_exist} | {n_non_exist} | {m_exist:.4f} | {m_non_exist:.4f} | {pval:.4f} | {is_sig}|")
            
            if is_sig:
                significant_cases.append({
                    'type': 'Quantile',
                    'sdg': f"{sdg_name} ({sdg_id})",
                    'target': target_label,
                    'quantile': f"{quantile_label} (Group {q})",
                    'p_value': pval,
                    'mean_diff': m_exist - m_non_exist
                })
        print("\n")

    run_quantile_analysis('scope_1_revenue', 'target_scope_1', 'target scope 1', 'scope 1 revenue quantile')
    run_quantile_analysis('scope_1_revenue', 'log_target_scope_1', 'target scope 1 log', 'scope 1 revenue quantile')
    run_quantile_analysis('scope_2_revenue', 'target_scope_2', 'target scope 2', 'scope 2 revenue quantile')
    run_quantile_analysis('scope_2_revenue', 'log_target_scope_2', 'target scope 2 log', 'scope 2 revenue quantile')
    
    print("-" * 50 + "\n")

print("=== Summary of Significant Cases ===")
if significant_cases:
    sig_df = pd.DataFrame(significant_cases)
    # Sort by p-value for better readability
    sig_df = sig_df.sort_values('p_value')
    print(sig_df.to_string(index=False))
else:
    print("No significant cases found.")


# 7. Experiments

To validate whether the insights from the earlier data analysis actually improve prediction of Scope 1 and Scope 2 emissions, several experiments were conducted. 
Each experiment combines a preprocessing strategy with a specific model type. The goal is to compare both datasets and modeling approaches in a controlled setting.

## Dataset

Two dataset configurations are compared.

### 1. Baseline Dataset
This follows the preprocessing pipeline originally provided by the Codeathon organizers. It includes:

- One-hot encoding of `region`  
- Adding sector-level revenue share using the revenue distribution table  
- Adding environmental activity score adjustments  
- Adding binary features representing Sustainable Development Goal (SDG) activities  
- Applying a variance threshold of 0.05 for feature selection  

### 2. Proposed Dataset
This dataset uses features identified through exploratory analysis and domain reasoning. The feature set includes:

- `log_total_revenue`
- `log_scope1_revenue`
- `log_scope2_revenue`
- `scope1_revenue_present` (0 or 1)
- `scope2_revenue_present` (0 or 1)
- `env_adjusted_score`
- `social_score`
- `governance_score`
- `country_code`  
  - Used directly as a categorical variable for tree-based models  
  - One-hot encoded for linear models  

The target variables (`target_scope_1`, `target_scope_2`) are log-transformed during training to stabilize variance. Predictions are converted back to raw scale for final evaluation.


## Models

Three different models are used to evaluate linear trends, robustness to outliers, and nonlinear relationships.

### 1. Linear Regression
This is the baseline model defined by the organizers. It optimizes MSE and serves as a reference point. Linear regression is sensitive to outliers and cannot capture nonlinearity.

### 2. Median Regression
Our primary metric is MAE, and secondary metrics are log MAE and log RMSE. Since all emphasize central tendency and robustness, median regression is appropriate. The regularization constant (alpha) is set to 0 to avoid biasing the solution.

### 3. CatBoost
Given the skewed numerical features and categorical variables such as `country_code`, a tree-based model is well suited. CatBoost handles categorical encoding internally and captures nonlinear effects efficiently, making it a strong candidate for small datasets.


## Experiment Settings

We evaluate four combinations of dataset and model:

1. **Baseline Dataset + Linear Regression**  
   This recreates the original baseline setup and serves as a comparison anchor.

2. **Baseline Dataset + Median Regression**  
   This tests whether replacing the baseline model with a MAE-oriented model improves performance under the same feature configuration.

3. **Proposed Dataset + Median Regression**  
   This evaluates whether the newly engineered features provide meaningful gains over the baseline features.

4. **Proposed Dataset + CatBoost**  
   This leverages a tree-based method to capture nonlinear patterns and directly handle categorical variables, aiming for the best overall performance.


## Results

The following table summarizes the cross-validated performance of all four experiment settings.  
Metrics reported are MAE, log-MAE, and log-RMSE for both Scope 1 and Scope 2 targets.

### Final Results Table

| Model                                      | Scope 1 MAE | Scope 1 Log MAE | Scope 1 Log RMSE | Scope 2 MAE | Scope 2 Log MAE | Scope 2 Log RMSE |
|--------------------------------------------|-------------|-----------------|------------------|-------------|-----------------|------------------|
| LinearRegression + Baseline                | 64409.1131  | 2.5848          | 3.5073           | 71708.9193  | 3.1736          | 4.3514           |
| MedianRegression + Baseline                | 51830.9031  | 2.0765          | 2.8194           | 52615.5835  | 2.1804          | 3.2238           |
| MedianRegression + NewFeatures + LogTarget | 51025.5594  | 1.5479          | 1.9553           | 55099.4216  | 1.8250          | **2.4967**       |
| CatBoost + NewFeatures + LogTarget         | **50094.6028** | **1.4946**     | **1.8922**       | **52235.8818** | **1.8224**     | 2.5350       |


## Interpretation

### 1. Baseline vs Median Regression  
- Median regression significantly outperforms linear regression across all metrics.  
- This confirms that MAE-oriented modeling and robustness to outliers are important for this task.

### 2. Baseline Features vs Proposed Features  
- Replacing the baseline features with the proposed analytical features further improves performance.  
- Log-transforming the targets stabilizes heavy skewness and reduces both log-MAE and log-RMSE substantially.

### 3. CatBoost Performance  
- CatBoost delivers the best results for Scope 1 and competitive results for Scope 2.  
- Its ability to handle nonlinearity and categorical variables directly gives it an advantage.  
- Overall, **CatBoost + Proposed Features + Log Target** is the top-performing setup.


## Summary

- **Feature engineering** based on domain insights leads to consistent improvements.  
- **Log-target modeling** is highly effective for emission data with heavy-tailed distributions.  
- **CatBoost** is the strongest model overall, indicating that nonlinear relationships and categorical structure play an important role in predicting emissions.



In [ ]:
import pandas as pd
import json
from src.preprocessor import (
    CodeathonBaselinePreprocessor,
    ProposedFeaturePreprocessor
)
from src.models import (
    LinearRegressionModel,
    MedianRegressionModel,
    LogTargetCatBoostModel,
    LogTargetMedianRegressionModel
)
from src.trainer import Trainer
from tabulate import tabulate


# -----------------------------------------------------
# Load Data
# -----------------------------------------------------
train_df = pd.read_csv("data/train.csv")
y_scope1 = train_df['target_scope_1']
y_scope2 = train_df['target_scope_2']

# -----------------------------------------------------
# Define Experiment Configurations
# -----------------------------------------------------
experiments = [
    {
        "name": "LinearRegression + Baseline",
        "preprocessor": CodeathonBaselinePreprocessor(),
        "model_class": LinearRegressionModel,
        "model_params": {},
        "trainer_params": {"n_splits": 5, "random_state": 42},
    },
    {
        "name": "MedianRegression + Baseline",
        "preprocessor": CodeathonBaselinePreprocessor(),
        "model_class": MedianRegressionModel,
        "model_params": {},
        "trainer_params": {"n_splits": 5, "random_state": 42},
    },
    {
        "name": "MedianRegression + NewFeatures + LogTarget",
        "preprocessor": ProposedFeaturePreprocessor(),
        "model_class": LogTargetMedianRegressionModel,
        "model_params": {},
        "trainer_params": {"n_splits": 5, "random_state": 42},
    },
    {
        "name": "CatBoost + NewFeatures + LogTarget",
        "preprocessor": ProposedFeaturePreprocessor(tree=True),
        "model_class": LogTargetCatBoostModel,
        "model_params": {
            'iterations': 100,
            'depth': 6,
            'learning_rate': 0.1,
            'random_state': 42
        },
        "trainer_params": {"n_splits": 5, "random_state": 42},
    },
]

# -----------------------------------------------------
# Run Experiments
# -----------------------------------------------------
results = []

for exp in experiments:
    print(f"\n=== Running {exp['name']} ===")

    # Preprocess
    X = exp["preprocessor"].fit_transform(train_df)

    # Trainer
    trainer = Trainer(
        model_class=exp["model_class"],
        model_params=exp["model_params"],
        **exp["trainer_params"]
    )

    # Train scope 1
    m1 = trainer.train(X, y_scope1)
    # Train scope 2
    m2 = trainer.train(X, y_scope2)

    # Save results into table row
    results.append({
        "model": exp["name"],
        "scope1_mae": m1["mean_mae"],
        "scope1_log_mae": m1["mean_log_mae"],
        "scope1_log_rmse": m1["mean_log_rmse"],
        "scope2_mae": m2["mean_mae"],
        "scope2_log_mae": m2["mean_log_mae"],
        "scope2_log_rmse": m2["mean_log_rmse"],
    })

# -----------------------------------------------------
# Results DataFrame
# -----------------------------------------------------
results_df = pd.DataFrame(results)
print("\n\n==================== Final Results Table ====================")
print(
    tabulate(
        results_df,
        headers="keys",
        tablefmt="github",
        floatfmt=".4f",
        showindex=False
    )
)



# 8. CatBoost Hyperparameter Optimization

This section runs a lightweight hyperparameter search to improve the CatBoost model using the proposed feature set and log-transformed targets.

## Overview

- The proposed feature preprocessor is applied once (`tree=True`) and reused.
- A simple **random search** with 20 trials is performed.
- Each trial trains two CatBoost models (Scope 1, Scope 2) using 5-fold CV.
- Optimization target is the average MAE across both scopes.

## Search Space

- `iterations`: 200–1000  
- `depth`: 4–8  
- `learning_rate`: 0.03–0.1  
- `l2_leaf_reg`: 1–10  
- `random_state`: 42  

## Process

For each trial:

1. Sample hyperparameters  
2. Train CatBoost (log-target mode)  
3. Compute MAE for Scope 1 and Scope 2  
4. Calculate `(MAE1 + MAE2) / 2`  
5. Record the result  

The best configuration is chosen by sorting on the combined MAE.

## Final Step

- Retrain CatBoost with the best hyperparameters  
- Add results as a new row:  
  **"CatBoost + NewFeatures + LogTarget (Optimized)"**  
- Append to the previous experiment table


# Results (Including Optimized CatBoost)

### Final Results Table (With Optimization)

| Model                                          | Scope 1 MAE | Scope 1 Log MAE | Scope 1 Log RMSE | Scope 2 MAE | Scope 2 Log MAE | Scope 2 Log RMSE |
|------------------------------------------------|-------------|-----------------|------------------|-------------|-----------------|------------------|
| LinearRegression + Baseline                    | 64409.1131  | 2.5848          | 3.5073           | 71708.9193  | 3.1736          | 4.3514           |
| MedianRegression + Baseline                    | 51830.9031  | 2.0765          | 2.8194           | 52615.5835  | 2.1804          | 3.2238           |
| MedianRegression + NewFeatures + LogTarget     | 51025.5594  | 1.5479          | 1.9553           | 55099.4216  | 1.8250          | 2.4967           |
| CatBoost + NewFeatures + LogTarget             | 50094.6028  | **1.4946**      | 1.8922           | **52235.8818** | 1.8224        | 2.5350           |
| **CatBoost + NewFeatures + LogTarget (Optimized)** | **49595.7693** | 1.4977      | **1.8918**       | 53014.5223  | **1.8212**      | **2.5132**       |

### Best Values by Metric

- **Scope 1 MAE:** Optimized CatBoost  
- **Scope 1 Log MAE:** CatBoost (non-optimized)  
- **Scope 1 Log RMSE:** Optimized CatBoost  
- **Scope 2 MAE:** CatBoost (non-optimized)  
- **Scope 2 Log MAE:** Optimized CatBoost  
- **Scope 2 Log RMSE:** Optimized CatBoost  


## Summary

- CatBoost consistently outperforms all linear and median regression baselines.
- Hyperparameter optimization **only provides minor improvement**, mainly in Scope 1 metrics.
- The non-optimized CatBoost model already performs extremely well across all metrics.
- Given the marginal gains and small dataset size, **extensive hyperparameter tuning is unnecessary** for this task.

In conclusion, **CatBoost + Proposed Features + LogTarget** is sufficiently strong even without hyperparameter optimization.


In [ ]:
import numpy as np
import pandas as pd
import random
from src.preprocessor import ProposedFeaturePreprocessor
from src.models import LogTargetCatBoostModel
from src.trainer import Trainer
from tabulate import tabulate


# -----------------------------------------------------
# Settings
# -----------------------------------------------------
N_TRIALS = 20
RANDOM_STATE = 42

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

train_df = pd.read_csv("data/train.csv")
y_scope1 = train_df["target_scope_1"]
y_scope2 = train_df["target_scope_2"]

# -----------------------------------------------------
# Preprocess once (tree mode)
# -----------------------------------------------------
preprocessor = ProposedFeaturePreprocessor(tree=True)
X = preprocessor.fit_transform(train_df)

# -----------------------------------------------------
# Hyperparameter Search Space
# -----------------------------------------------------
def sample_params():
    return {
        "iterations": random.choice([200, 300, 500, 800, 1000]),
        "depth": random.choice([4, 5, 6, 7, 8]),
        "learning_rate": random.choice([0.03, 0.05, 0.07, 0.1]),
        "l2_leaf_reg": random.choice([1, 3, 5, 7, 10]),
        "random_state": RANDOM_STATE
    }


# -----------------------------------------------------
# Optimization Loop
# -----------------------------------------------------
trial_results = []

print("\n=== CatBoost Hyperparameter Optimization ===")

for trial in range(1, N_TRIALS + 1):
    params = sample_params()

    trainer = Trainer(
        model_class=LogTargetCatBoostModel,
        model_params=params,
        n_splits=5,
        random_state=RANDOM_STATE
    )

    m1 = trainer.train(X, y_scope1)
    m2 = trainer.train(X, y_scope2)

    combined_score = (m1["mean_mae"] + m2["mean_mae"]) / 2

    trial_results.append({
        "trial": trial,
        **params,
        "scope1_mae": m1["mean_mae"],
        "scope2_mae": m2["mean_mae"],
        "combined": combined_score
    })

    print(f"Trial {trial}/{N_TRIALS}: combined MAE = {combined_score:.3f} | params = {params}")


# -----------------------------------------------------
# Results data frame
# -----------------------------------------------------
hpo_results_df = pd.DataFrame(trial_results).sort_values("combined")

print("\n\n==================== Optimization Results ====================")
print(
    tabulate(
        hpo_results_df,
        headers="keys",
        tablefmt="github",
        floatfmt=".4f",
        showindex=False
    )
)

# -----------------------------------------------------
# Extract best params
# -----------------------------------------------------
best_row = hpo_results_df.iloc[0].to_dict()
best_params = {
    key: best_row[key]
    for key in ["iterations", "depth", "learning_rate", "l2_leaf_reg", "random_state"]
}

print("\nBest Parameters:")
print(best_params)


# =====================================================
# Retrain CatBoost using best params
# =====================================================
trainer = Trainer(
    model_class=LogTargetCatBoostModel,
    model_params=best_params,
    n_splits=5,
    random_state=RANDOM_STATE
)

m1_best = trainer.train(X, y_scope1)
m2_best = trainer.train(X, y_scope2)

# Row to append
optimized_row = {
    "model": "CatBoost + NewFeatures + LogTarget (Optimized)",
    "scope1_mae": m1_best["mean_mae"],
    "scope1_log_mae": m1_best["mean_log_mae"],
    "scope1_log_rmse": m1_best["mean_log_rmse"],
    "scope2_mae": m2_best["mean_mae"],
    "scope2_log_mae": m2_best["mean_log_mae"],
    "scope2_log_rmse": m2_best["mean_log_rmse"],
}


# =====================================================
# Append to previous cell's results_df
# =====================================================
try:
    final_results_df = pd.concat(
        [results_df, pd.DataFrame([optimized_row])],
        ignore_index=True
    )

    print("\n\n==================== Final Results Table (With Optimization) ====================")
    print(
        tabulate(
            final_results_df,
            headers="keys",
            tablefmt="github",
            floatfmt=".4f",
            showindex=False
        )
    )

except NameError:
    print("\n[WARNING] previous cell's results_df not found.")
    print("Optimized row:")
    print(optimized_row)


# 9. Final Prediction (submission.csv)

In [ ]:
from src.preprocessor import ProposedFeaturePreprocessor
from src.models import LogTargetCatBoostModel
import pandas as pd
import os

print("\n--- Training Final Models and Generating Submission ---")

# Load data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Initialize preprocessor
preprocessor = ProposedFeaturePreprocessor(tree=True)

# Fit on train and transform
X = preprocessor.fit_transform(train_df)
X_test = preprocessor.transform(test_df)

# Extract targets
y_scope1 = train_df['target_scope_1']
y_scope2 = train_df['target_scope_2']

# Model params (from final.ipynb:L49-L60)
model_params = {
    'iterations': 100,
    'depth': 6,
    'learning_rate': 0.1,
    'random_state': 42
}

# Train Scope 1 on full data
print("Training Scope 1...")
model_s1 = LogTargetCatBoostModel(**model_params)
model_s1.fit(X, y_scope1)
s1_predictions = model_s1.predict(X_test)

# Train Scope 2 on full data
print("Training Scope 2...")
model_s2 = LogTargetCatBoostModel(**model_params)
model_s2.fit(X, y_scope2)
s2_predictions = model_s2.predict(X_test)

# Create submission DataFrame
submission = pd.DataFrame({
    'entity_id': test_df['entity_id'],
    's1_predictions': s1_predictions,
    's2_predictions': s2_predictions
})

# Save submission
submission.to_csv("submission.csv", index=False)
print("Submission saved to submission.csv")


# 10. Conclusion